In [1]:
prefix = '/home/ines/repositories/'
# prefix = '/Users/ineslaranjeira/Documents/Repositories/'

In [2]:
""" 
IMPORTS
"""
import os
import numpy as np
import pickle
import pandas as pd
from one.api import ONE
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore
from matplotlib.colors import ListedColormap

# Get my functions
from functions import idxs_from_files, state_identifiability, align_bin_design_matrix, states_per_trial_phase, broader_label
from functions import define_trial_types, rescale_sequence, plot_binned_sequence

one = ONE(mode='remote')

# Parameters

In [3]:
# LOAD DATA
data_path = prefix + 'representation_learning_variability/paper-individuality/data/design_matrices/'
paw_wavelet_path = prefix + 'representation_learning_variability/paper-individuality/data/paw_wavelets/'
wheel_wavelet_path = prefix + 'representation_learning_variability/paper-individuality/data/wheel_wavelets/'

all_files = os.listdir(data_path)
design_matrices = [item for item in all_files if 'design_matrix' in item and 'standardized' not in item]
idxs, mouse_names = idxs_from_files(design_matrices)

""" FITTING PARAMETERS """
num_iters = 100
num_states = 2
num_train_batches = 5
method='prior'
fit_method='em'
threshold = 0


optimal_k = 8
optimal_k_wheel = 3

whisker_params = str(num_train_batches)+'_'+method+'_'+fit_method+'_zsc_'+'True/'
licking_params = str(num_train_batches)+'_'+method+'_'+fit_method+'_zsc_'+'False/'

states_path = prefix + 'representation_learning_variability/paper-individuality/data/hmm/most_likely_states/'
paw_wavelet_states_path = prefix + 'representation_learning_variability/paper-individuality/data/paw_most_likely_states/'
wheel_wavelet_states_path = prefix + 'representation_learning_variability/paper-individuality/data/wheel_most_likely_states/'


# Individual sessions

In [4]:
# Identify sessions available to process
sessions_to_process = []
for m, mat in enumerate(idxs):
    mouse_name = mat[37:]
    session = mat[:36]
    fit_id = str(mouse_name + session)
    whisker_filename = os.path.join(states_path + whisker_params, 'whisker_me' + '_' + fit_id)
    licks_filename = os.path.join(states_path + licking_params, 'Lick count' + '_' + fit_id)
    paw_wavelet_filename = os.path.join(paw_wavelet_states_path, "most_likely_states_" + str(optimal_k) + '_' + fit_id+'.npy')
    wheel_wavelet_filename = os.path.join(wheel_wavelet_states_path, "most_likely_states_" + str(optimal_k_wheel) + '_' + fit_id+'.npy')

    # if os.path.exists(whisker_filename) and os.path.exists(licks_filename) and os.path.exists(paw_wavelet_filename) and os.path.exists(wheel_wavelet_filename):
    if os.path.exists(whisker_filename) and os.path.exists(licks_filename) and os.path.exists(paw_wavelet_filename):
        sessions_to_process.append((mouse_name, session))

print(f"Found {len(sessions_to_process)} sessions to process.")

Found 218 sessions to process.


In [5]:
identifiable_mapping = {}

counter = 0
for wheel in [0, 1, 2]:
    for lick in [0, 1]:
        for whisk in [0, 1]:
            for paw in range(8):
                key = f"{paw}{whisk}{lick}{wheel}"
                identifiable_mapping[key] = float(counter)
                counter += 1

identifiable_mapping["nan"] = np.nan

paw_fix_mapping = {0:4, 1:1, 2:5, 3:7, 4:6, 5:2, 6:0, 7:3}
wheel_fix_mapping = {0:1, 1:0, 2:2} 

In [6]:
def wheel_over_wavelet_clusters(init, inter, empirical_data, session_trials, color_states):
    # Plot raw trace over states
    # init should be in seconds; inter should be in frames
    frame_rate = 60
    plot_min = -10
    plot_max = 10

    fig, ax = plt.subplots(ncols=1 , nrows=1, sharex=False, sharey=False, figsize=[20, 5])
    plt.rc('font', size=12)
    palette = ListedColormap(sns.color_palette('Set2', n_colors=8))

    # ax.plot(data)
    ax.plot(empirical_data.loc[empirical_data['Bin']>=init, 'avg_wheel_vel'].reset_index(), color='black')
    ax.plot(zscore(empirical_data.loc[empirical_data['Bin']>=init, 'l_paw_x'].reset_index(), nan_policy='omit'), color='green')
    ax.plot(zscore(empirical_data.loc[empirical_data['Bin']>=init, 'r_paw_x'].reset_index(), nan_policy='omit'), color='red')
    ax.imshow(np.concatenate([empirical_data.loc[empirical_data['Bin']>=init, color_states]])[None,:],
                extent=(0, len(np.concatenate([empirical_data.loc[empirical_data['Bin']>=init, color_states]])), -10, 10),
                aspect="auto",
                cmap=palette,
                alpha=0.6)

    for state, marker in zip([0.,1.], ['o','']):
        mask_whisker = empirical_data.loc[empirical_data['Bin']>=init, 'whisker_me_states'] == state
        masked_time_whisker = (empirical_data.loc[empirical_data['Bin']>=init, 'Bin'][mask_whisker].reset_index()['Bin']* frame_rate)-init*frame_rate
        mask_lick = empirical_data.loc[empirical_data['Bin']>=init, 'Lick count_states'] == state
        masked_time_lick = (empirical_data.loc[empirical_data['Bin']>=init, 'Bin'][mask_lick].reset_index()['Bin']* frame_rate)-init*frame_rate
        ax.scatter(masked_time_whisker, np.zeros(np.sum(mask_whisker))+2, marker=marker, c='purple', s=20, alpha=.6)
        ax.scatter(masked_time_lick, np.zeros(np.sum(mask_lick))+3, marker=marker, c='blue', s=20, alpha=.6)
    

    ax.vlines(np.array(session_trials['goCueTrigger_times'] -init)*frame_rate, plot_min, plot_max, label='Stim On', 
                color='Black', linewidth=2)
    ax.vlines(np.array(session_trials.loc[session_trials['feedbackType']==1, 'feedback_times'] * frame_rate)-init*frame_rate, 
                plot_min, plot_max, label='Correct', color='Green', linewidth=2)
    ax.vlines(np.array(session_trials.loc[session_trials['feedbackType']==-1, 'feedback_times'] * frame_rate)-init*frame_rate, 
                plot_min, plot_max, label='Incorrect', color='Red', linewidth=2)
    ax.vlines(np.array(session_trials['firstMovement_times'] * frame_rate)-init*frame_rate, plot_min, plot_max, label='First movement', color='Blue')
    ax.vlines(np.array(session_trials['intervals_0'] * frame_rate)-init*frame_rate, plot_min, plot_max, label='Trial end', color='Grey', linewidth=2)
    ax.vlines(np.array((session_trials['goCueTrigger_times'] - session_trials['quiescencePeriod']) * frame_rate)-init*frame_rate, 
                plot_min, plot_max, label='Quiescence start', color='Pink', linewidth=2)

    ax.set_xlim([init, init+inter])
    ax.set_ylabel("Wheel velocity")
    ax.set_xlabel("Time (s)")
    ax.set_xticks(np.arange(0, inter, inter/5),np.arange(init, 
                                                          (init+inter)/frame_rate, (inter/frame_rate)/5))
    ax.set_title("Wavelet transform clusters")
    ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
    ax.set_ylim([-10, 10])

    plt.tight_layout()    
    plt.show()

In [ ]:
var_interest = ['paw', 'whisker_me', 'Lick count', 'wheel']
path_sets = [paw_wavelet_states_path, states_path + whisker_params, states_path+licking_params, wheel_wavelet_states_path]

states_trial_type = pd.DataFrame(columns=['mouse_name', 'session', 'correct', 'choice', 'contrast', 
                                          'reaction', 'response', 'elongation', 'most_likely_states', 
                                          'identifiable_states', 'Bin', 'label'])

for m, mat in enumerate(sessions_to_process):

    mouse_name = mat[0]
    session = mat[1]

    fit_id = str(mouse_name+session)
    trials_file = data_path + "session_trials_" + str(session) + '_'  + mouse_name
    session_trials = pd.read_parquet(trials_file, engine='pyarrow').reset_index() 
    
    session_states = pd.DataFrame()
    for v, var in enumerate(var_interest):

        # Get most likely states filename
        use_path = path_sets[v]
        var_name = var_interest[v]
        use_k = optimal_k if var=='paw' else optimal_k_wheel
        states_filename = os.path.join(use_path, 
                                       f"{'most_likely_states_'+str(use_k)+ '_' + fit_id + '.npy' if var_name=='paw' or var_name=='wheel' else var_name+ '_' + fit_id}")
        
        column_name = var + '_states'
        # Load design matrix used to obtain states and compare sizes
        if var == 'wheel':
            use_vars = ['avg_wheel_vel0.5', 'avg_wheel_vel1.0', 'avg_wheel_vel2.0', 'avg_wheel_vel4.0', 'avg_wheel_vel8.0', 
                            'avg_wheel_vel16.0', 'avg_wheel_vel32.0']   
            filename = wheel_wavelet_path + "wheel_vel_wavelets_" + str(session) + '_'  + mouse_name
            design_matrix = pd.read_parquet(filename)
            most_likely_states = np.load(open(states_filename, "rb"))
            # Rename states to be more intuitive
            replace_func = np.vectorize(wheel_fix_mapping.get)
            most_likely_states = replace_func(most_likely_states)
            # not_nan_indices = ~np.isnan(np.array(design_matrix)).any(axis=1)

            not_nan_indices = ~np.isnan(np.array(design_matrix[use_vars])).any(axis=1)
            assert np.sum(not_nan_indices) == len(most_likely_states)
            design_matrix[column_name] = design_matrix['Bin'] * np.nan
            design_matrix[column_name].loc[not_nan_indices] = most_likely_states
        
        elif var == 'paw':
            use_vars = ['l_paw_x0.5', 'l_paw_x1.0', 'l_paw_x2.0', 'l_paw_x4.0', 'l_paw_x8.0', 
                    'l_paw_y0.5', 'l_paw_y1.0', 'l_paw_y2.0', 'l_paw_y4.0', 'l_paw_y8.0', 
                    'r_paw_x0.5', 'r_paw_x1.0', 'r_paw_x2.0', 'r_paw_x4.0', 'r_paw_x8.0', 
                    'r_paw_y0.5', 'r_paw_y1.0', 'r_paw_y2.0', 'r_paw_y4.0', 'r_paw_y8.0']
            filename = paw_wavelet_path + "paw_vel_wavelets_" + str(session) + '_'  + mouse_name
            design_matrix = pd.read_parquet(filename)
            most_likely_states = np.load(open(states_filename, "rb"))
            # Rename states to be more intuitive
            replace_func = np.vectorize(paw_fix_mapping.get)
            most_likely_states = replace_func(most_likely_states)
            not_nan_indices = ~np.isnan(np.array(design_matrix[use_vars])).any(axis=1)
            assert np.sum(not_nan_indices) == len(most_likely_states)
            design_matrix[column_name] = design_matrix['Bin'] * np.nan
            design_matrix[column_name].loc[not_nan_indices] = most_likely_states

        else:
            most_likely_states, _, _ = pickle.load(open(states_filename, "rb"))
            filename = paw_wavelet_path + "paw_vel_wavelets_" + str(session) + '_'  + mouse_name
            design_matrix = pd.read_parquet(filename)
            not_nan_indices = ~np.isnan(np.array(design_matrix)).any(axis=1)
            num_timesteps = np.shape(design_matrix.dropna())[0]
            hmm_length = (num_timesteps // num_train_batches) * num_train_batches
            assert hmm_length == len(most_likely_states)
            use_indices = not_nan_indices[:hmm_length]
            design_matrix[column_name] = design_matrix['Bin'] * np.nan
            rows = design_matrix.index[not_nan_indices][:hmm_length]
            design_matrix.loc[rows, column_name] = most_likely_states

        # Join data
        if v == 0:
            session_states = design_matrix[['Bin', 'Lick count', 'avg_wheel_vel', 'whisker_me', 'avg_whisker_me',
       'l_paw_x', 'l_paw_y', 'r_paw_x', 'r_paw_y', column_name]]
        else:
            session_states = session_states.merge(design_matrix[['Bin', column_name]], on='Bin', how='outer')
    
    # Transform states into identifiable states
    sets_to_identify = ['whisker_me', 'Lick count']
    session_states = state_identifiability(session_states, sets_to_identify)

    # Combine states
    session_states = session_states.dropna() 
    c_states = np.array(session_states[['paw_states', 'whisker_me_states', 'Lick count_states', 'wheel_states']].astype(int))
    combined_states = np.array([''.join(map(str, row)) for row in c_states])
    session_states['identifiable_states'] = combined_states

    # Back to integer
    replace_func = np.vectorize(identifiable_mapping.get)
    integer_states = replace_func(combined_states)
    session_states['most_likely_states'] = integer_states
    
    # Get back the NaNs
    final_states = design_matrix[['Bin']].merge(session_states, on='Bin', how='outer')
    
    # Align bins
    multiplier = 1
    event_type_list = ['goCueTrigger_times']  # , 'feedback_times', 'firstMovement_times'
    event_type_name = ['Go cue']  # , 'Feedback time', 'First movement onset'
    init = -1 * multiplier
    end = 1.5 * multiplier
    empirical_data = align_bin_design_matrix(init, end, event_type_list, session_trials, final_states, final_states['most_likely_states'], multiplier)
    empirical_data = empirical_data.drop(columns=['new_bin'])
    
    """ Trial types """
    # Split in Foundtrial types
    states_trial = states_per_trial_phase(empirical_data, session_trials, multiplier)
    states_trial['mouse_name'] = mouse_name
    states_trial['session'] = session
    states_trial = broader_label(states_trial)
    states_trial = states_trial[['Bin', 'Lick count', 'avg_wheel_vel', 'whisker_me', 
                                'l_paw_x', 'l_paw_y', 'r_paw_x', 'r_paw_y', 'most_likely_states',
                                'correct', 'choice', 'contrast', 'block', 'reaction', 'response',
                                'elongation', 'wsls', 'trial_id', 'goCueTrigger_times',
                                'identifiable_states', 'label', 'mouse_name', 'session',
                                'broader_label']]
    
    # Save to big df
    states_trial_type = pd.concat([states_trial_type, states_trial], ignore_index=True)
    del states_trial

In [8]:
states_trial_type.to_parquet('states_trial_type_wheel_02-16-2026', compression='gzip')  

## Trial epoch barcoding

In [8]:
states_trial_type = pd.read_parquet('states_trial_type_wheel_02-16-2026')

In [ ]:
trial_type_agg = ['correct_str', 'contrast_str', 'block_str', 'choice']

plot = False
target_length = 10

all_sequences = pd.DataFrame(columns=['mouse_name', 'trial_type', 'broader_label', 'binned_sequence'])
for m, mat in enumerate(sessions_to_process):

    mouse_name = mat[0]
    session = mat[1] 
    fit_id = str(mouse_name+session)

    states_trial = states_trial_type.loc[states_trial_type['session']==session]
    num_states = len(states_trial_type['most_likely_states'].unique())

    """ Define trial types  """
    states_df = define_trial_types(states_trial, trial_type_agg)
    vars = ['sample', 'trial_type', 'broader_label', 'mouse_name', 'most_likely_states']

    df_grouped = states_df.groupby(vars[:-1])['most_likely_states'].apply(list).reset_index()
    df_grouped.rename(columns={'most_likely_states': 'sequence'}, inplace=True)
    df_grouped['binned_sequence'] = df_grouped['sequence'].apply(lambda seq: rescale_sequence(seq, target_length))  # New function removed NaNs
    
    if plot == True:
        states_to_append = np.arange(0, num_states)
        for i in range(100):
            if df_grouped['broader_label'][i] == 'Quiescence':
                plot_binned_sequence(df_grouped, i, states_to_append, 'viridis')
    
    all_sequences = pd.concat([all_sequences, df_grouped[['mouse_name', 'sample', 'trial_type', 'broader_label', 'binned_sequence']]], ignore_index=True)
    

In [10]:
all_sequences.to_parquet('all_sequences_02-16-2026', compression='gzip')  

## Raw data sequences

In [11]:
from scipy.stats import mode


In [12]:
def rescale_sequence(seq, target_length, estimator):
    """
    Rescales a categorical sequence to a fixed target length.
    
    - If `target_length` is smaller than the original length, it takes the mode of each bin.
    - If `target_length` is larger, it repeats values evenly.
    
    Parameters:
        seq (array-like): The original categorical sequence.
        target_length (int): The desired length of the output sequence.
    
    Returns:
        np.ndarray: The transformed sequence with the specified target length.
    """
    original_length = len(seq)

    if original_length == target_length:
        return np.array(seq)  # No change needed

    if target_length < original_length:
        # Compression: Split into bins and take mode of each bin
        bins = np.array_split(seq, target_length)
        # return np.array([mode(b)[0][0] for b in bins])  # Extract mode from result
        if estimator == 'mode':
            result = np.array([mode(b)[0] for b in bins])
        elif estimator == 'mean':
            result = np.array([np.mean(b) for b in bins])
        return result  

    else:
        # Stretching: Repeat values to fit new size
        stretched_indices = np.floor(np.linspace(0, original_length - 1, target_length)).astype(int)
        return np.array(seq)[stretched_indices]  # Map stretched indices to original values

In [66]:
def plot_binned_sequence(df_grouped, index, states_to_append, var_name, palette):
        title = df_grouped['broader_label'][index]
        fig, axs = plt.subplots(2, 1, sharex=False, sharey=True, figsize=(5, 2))
        axs[0].imshow(np.concatenate([df_grouped[var_name+'_sequence'][index], states_to_append])[None,:],  
                extent=(0, len(np.concatenate([df_grouped[var_name+'_sequence'][index], states_to_append])), 
                        0, 1),
                aspect="auto",
                cmap=palette,
                alpha=0.7) 
        axs[0].set_xlim([0, len(df_grouped[var_name+'_sequence'][index])])

        axs[1].imshow(np.concatenate([df_grouped[var_name+'_binned_sequence'][index], states_to_append])[None,:],  
                extent=(0, len(np.concatenate([df_grouped[var_name+'_binned_sequence'][index], states_to_append])), 
                        0, 1),
                aspect="auto",
                cmap=palette,
                alpha=0.7) 
        axs[1].set_xlim([0, len(df_grouped[var_name+'_binned_sequence'][index])])
        axs[0].set_title(title)
        plt.tight_layout()

In [ ]:
trial_type_agg = ['correct_str', 'contrast_str', 'block_str', 'choice']

plot = False
target_length = 10

all_sequences = pd.DataFrame()
for m, mat in enumerate(sessions_to_process):

    mouse_name = mat[0]
    session = mat[1] 
    fit_id = str(mouse_name+session)

    states_trial = states_trial_type.loc[states_trial_type['session']==session]
    num_states = len(states_trial_type['most_likely_states'].unique())

    """ Define trial types  """
    states_df = define_trial_types(states_trial, trial_type_agg)
    vars = ['sample', 'trial_type', 'broader_label', 'mouse_name', 'most_likely_states']
    
    """ Pre-process data  """
    states_df['whisker_me'] = zscore(np.array(states_df['whisker_me']), 
                                                    nan_policy='omit', axis=0)
    states_df['l_paw_x_vel'] = states_df['l_paw_x'] * np.nan
    states_df['l_paw_y_vel'] = states_df['l_paw_x'] * np.nan
    states_df['r_paw_x_vel'] = states_df['l_paw_x'] * np.nan
    states_df['r_paw_y_vel'] = states_df['l_paw_x'] * np.nan
    
    states_df['l_paw_x_vel'][1:] =zscore(np.diff(states_df['l_paw_x']/2), 
                                                    nan_policy='omit', axis=0)
    states_df['l_paw_y_vel'][1:] = zscore(np.diff(states_df['l_paw_y']/2), 
                                                    nan_policy='omit', axis=0)
    states_df['r_paw_x_vel'][1:] = zscore(np.diff(states_df['r_paw_x']), 
                                                    nan_policy='omit', axis=0)
    states_df['r_paw_y_vel'][1:] = zscore(np.diff(states_df['r_paw_y']), 
                                                    nan_policy='omit', axis=0)

    process_vars = ['Lick count', 'whisker_me', 'l_paw_x_vel', 'l_paw_y_vel', 'r_paw_x_vel', 'r_paw_y_vel']
    all_vars_grouped = pd.DataFrame()
    
    for v, variable in enumerate(process_vars):
        
        df_grouped = states_df.groupby(vars[:-1])[variable].apply(list).reset_index()
        df_grouped.rename(columns={variable: variable+'_sequence'}, inplace=True)
        if v == 0:
            df_grouped[variable+'_binned_sequence'] = df_grouped[variable+'_sequence'].apply(lambda seq: rescale_sequence(seq, target_length, 'mode'))  # New function removed NaNs
            all_vars_grouped = df_grouped.copy()
        else:
            df_grouped[variable+'_binned_sequence'] = df_grouped[variable+'_sequence'].apply(lambda seq: rescale_sequence(seq, target_length, 'mean'))  # New function removed NaNs
            all_vars_grouped = all_vars_grouped.merge(df_grouped, on=['sample', 'trial_type', 'broader_label', 'mouse_name'])
        all_vars_grouped = all_vars_grouped.drop(columns=[variable+'_sequence'])
            
        # if plot == True:
        #     states_to_append = np.arange(0, num_states)
        #     for i in range(10):
        #         if df_grouped['broader_label'][i] == 'Choice':
        #             plot_binned_sequence(df_grouped, i, states_to_append, variable, 'viridis')
    
    all_sequences = pd.concat([all_sequences, all_vars_grouped], ignore_index=True)

## Wheel syllable binning

In [ ]:
trial_type_agg = ['correct_str', 'contrast_str', 'block_str', 'choice']

plot = False
target_length = 10

all_sequences = pd.DataFrame()
for m, mat in enumerate(sessions_to_process):

    mouse_name = mat[0]
    session = mat[1] 
    fit_id = str(mouse_name+session)

    states_trial = states_trial_type.loc[states_trial_type['session']==session]
    num_states = len(states_trial_type['most_likely_states'].unique())

    """ Define trial types  """
    states_df = define_trial_types(states_trial, trial_type_agg)
    vars = ['sample', 'trial_type', 'broader_label', 'mouse_name', 'most_likely_states']
    
    """ Pre-process data  """
    states_df['avg_wheel_vel'] = np.array(np.abs(states_df['avg_wheel_vel']))
    # states_df['l_paw_x_vel'] = states_df['l_paw_x'] * np.nan
    # states_df['l_paw_y_vel'] = states_df['l_paw_x'] * np.nan
    # states_df['r_paw_x_vel'] = states_df['l_paw_x'] * np.nan
    # states_df['r_paw_y_vel'] = states_df['l_paw_x'] * np.nan
    
    # states_df['l_paw_x_vel'][1:] =zscore(np.diff(states_df['l_paw_x']/2), 
    #                                                 nan_policy='omit', axis=0)
    # states_df['l_paw_y_vel'][1:] = zscore(np.diff(states_df['l_paw_y']/2), 
    #                                                 nan_policy='omit', axis=0)
    # states_df['r_paw_x_vel'][1:] = zscore(np.diff(states_df['r_paw_x']), 
    #                                                 nan_policy='omit', axis=0)
    # states_df['r_paw_y_vel'][1:] = zscore(np.diff(states_df['r_paw_y']), 
    #                                                 nan_policy='omit', axis=0)

    process_vars = ['avg_wheel_vel']
    all_vars_grouped = pd.DataFrame()
    
    for v, variable in enumerate(process_vars):
        
        df_grouped = states_df.groupby(vars[:-1])[variable].apply(list).reset_index()
        df_grouped.rename(columns={variable: variable+'_sequence'}, inplace=True)
        if v == 0:
            df_grouped[variable+'_binned_sequence'] = df_grouped[variable+'_sequence'].apply(lambda seq: rescale_sequence(seq, target_length, 'mode'))  # New function removed NaNs
            all_vars_grouped = df_grouped.copy()
        else:
            df_grouped[variable+'_binned_sequence'] = df_grouped[variable+'_sequence'].apply(lambda seq: rescale_sequence(seq, target_length, 'mean'))  # New function removed NaNs
            all_vars_grouped = all_vars_grouped.merge(df_grouped, on=['sample', 'trial_type', 'broader_label', 'mouse_name'])
        all_vars_grouped = all_vars_grouped.drop(columns=[variable+'_sequence'])
            
        # if plot == True:
        #     states_to_append = np.arange(0, num_states)
        #     for i in range(10):
        #         if df_grouped['broader_label'][i] == 'Choice':
        #             plot_binned_sequence(df_grouped, i, states_to_append, variable, 'viridis')
    
    all_sequences = pd.concat([all_sequences, all_vars_grouped], ignore_index=True)

In [15]:
all_sequences.to_parquet('all_sequences_wheel_10-07-2025', compression='gzip')  